In [3]:
!pip install pandas

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.3 MB 2.6 MB/s eta 0:00:05
   - -------------------------------------- 0.3/11.3 MB 3.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.3 MB 4.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/11.3 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.3 MB 6.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.3 MB 7.5 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.3 MB 8.1 MB/s eta 0:00:02
   ----------- ---------------------------- 3.2/11.3 MB 8.5 MB/s eta 0:00:01
   ------------- -------------------------- 3.8/11.3 MB 8.9 MB/s eta 0:00:01
   ---------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\doruk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

csv_path = "fire_nrt_J1V-C2_684138.csv"

cols_to_use = [
    "latitude", "longitude", "brightness", "scan", "track",
    "acq_date", "acq_time", "satellite", "instrument",
    "confidence", "version", "bright_t31", "frp", "daynight"
]

df = pd.read_csv(csv_path, usecols=cols_to_use)

df["acq_datetime"] = pd.to_datetime(
    df["acq_date"].astype(str) + " " + df["acq_time"].astype(str).str.zfill(4),
    format="%Y-%m-%d %H%M"
)

df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,acq_datetime
0,66.54968,76.69383,349.77,0.78,0.78,2025-11-03,102,N20,VIIRS,n,2.0NRT,264.76,12.22,N,2025-11-03 01:02:00
1,67.84235,79.97705,323.01,0.68,0.74,2025-11-03,102,N20,VIIRS,n,2.0NRT,252.59,2.41,N,2025-11-03 01:02:00
2,68.41514,83.62517,331.43,0.65,0.73,2025-11-03,102,N20,VIIRS,n,2.0NRT,259.89,6.94,N,2025-11-03 01:02:00
3,66.55634,76.68800,346.86,0.77,0.78,2025-11-03,102,N20,VIIRS,n,2.0NRT,261.19,12.22,N,2025-11-03 01:02:00
4,68.41590,83.63245,338.11,0.65,0.73,2025-11-03,102,N20,VIIRS,n,2.0NRT,259.79,7.29,N,2025-11-03 01:02:00


In [7]:
df = df[df["frp"] >= 20]
df = df[df["daynight"] == "D"]
df["lat_cell"] = (df["latitude"] / 0.2).round().astype(int)
df["lon_cell"] = (df["longitude"] / 0.2).round().astype(int)
df["date"] = df["acq_datetime"].dt.date
groups = df.groupby(["date", "lat_cell", "lon_cell"])

events = []

for (date, lat_c, lon_c), g in groups:
    lat_min = g["latitude"].min()
    lat_max = g["latitude"].max()
    lon_min = g["longitude"].min()
    lon_max = g["longitude"].max()
    buffer = 0.02
    lat_min -= buffer
    lat_max += buffer
    lon_min -= buffer
    lon_max += buffer
    t_start = datetime.combine(date, datetime.min.time()) - timedelta(days=1)
    t_end   = datetime.combine(date, datetime.max.time()) + timedelta(days=1)

    events.append({
        "bbox": [lon_min, lat_min, lon_max, lat_max],
        "time_interval": (
            t_start.strftime("%Y-%m-%d"),
            t_end.strftime("%Y-%m-%d")
        ),
        "firms_points": len(g)
    })

len(events)


6317

In [8]:
events = sorted(events, key=lambda e: e["firms_points"], reverse=True)
events = events[:80]

len(events)

80